<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import numpy as np
import pickle
pd.set_option('display.max_columns', None)
census = pd.read_csv('pop_unemploy_income.csv')
correct = pd.read_csv('zip_area_max.csv')
# crime = pd.read_csv('dataset_w_Zipcode.csv')

In [2]:
file = open("dataset_w_Zipcode.sav",'rb')
crime = pickle.load(file)
file.close()

In [3]:
census.drop(columns = ['Unnamed: 0'], inplace = True)
census.head()

,Year,zipcode,Total population,Unemployment_rate,Median household income
0,2021,90001,65673.0,12.698000,43494.637890
1,2021,90002,58419.0,14.501218,39275.637890
2,2021,90003,79775.0,15.416951,38721.312475
3,2021,90004,72373.0,11.519998,50624.674800
4,2021,90005,45600.0,11.622400,39509.847168


In [4]:
correct.zipcode = correct.zipcode.astype(int)
correct.head()

,zipcode,AREA_NAME,DR_NO
0,90001,77th Street,6275
1,90002,Southeast,30550
2,90003,Southeast,40160
3,90004,Olympic,38750
4,90005,Olympic,32468


In [5]:
census_g = census.merge(correct[['zipcode','AREA_NAME']], how = 'left', on = 'zipcode')
census_g

,Year,zipcode,Total population,Unemployment_rate,Median household income,AREA_NAME
0,2021,90001,65673.0,12.698000,43494.637890,77th Street
1,2021,90002,58419.0,14.501218,39275.637890,Southeast
2,2021,90003,79775.0,15.416951,38721.312475,Southeast
3,2021,90004,72373.0,11.519998,50624.674800,Olympic
4,2021,90005,45600.0,11.622400,39509.847168,Olympic
...,...,...,...,...,...,...
2815,2010,93544,1199.0,4.854425,39774.132044,Central
2816,2010,93550,85309.0,19.422680,31391.302043,N Hollywood
2817,2010,93551,57359.0,12.497155,67363.192943,Mission
2818,2010,93552,46003.0,16.167493,46173.248786,Devonshire


In [6]:
# find rows with no area names
print(census_g[census_g['AREA_NAME'].isnull()]['zipcode'].unique())
print(len(census_g[census_g['AREA_NAME'].isnull()]['zipcode']))
print(len(census_g))

[]
0
2820


In [7]:
# drop those rows
census_g.drop(census_g[census_g['AREA_NAME'].isnull()].index, inplace = True)
print(len(census_g))

2820


In [8]:
census_g.rename(columns={'Total population': 'Population', 'Median household income': 'Median_household_income'}, inplace=True)
census_g.head()

,Year,zipcode,Population,Unemployment_rate,Median_household_income,AREA_NAME
0,2021,90001,65673.0,12.698000,43494.637890,77th Street
1,2021,90002,58419.0,14.501218,39275.637890,Southeast
2,2021,90003,79775.0,15.416951,38721.312475,Southeast
3,2021,90004,72373.0,11.519998,50624.674800,Olympic
4,2021,90005,45600.0,11.622400,39509.847168,Olympic


In [9]:
# get total population for each area/year combination
pop = census_g.groupby(['AREA_NAME','Year'])[['Population']].sum().reset_index()
pop.head()

,AREA_NAME,Year,Population
0,77th Street,2010,620172.0
1,77th Street,2011,532941.0
2,77th Street,2012,532445.0
3,77th Street,2013,536127.0
4,77th Street,2014,539384.0


In [10]:
# get average income/unemployment rate for each area/year combination
income = census_g.groupby(['AREA_NAME','Year'])[['Unemployment_rate','Median_household_income']].mean().reset_index()
income

,AREA_NAME,Year,Unemployment_rate,Median_household_income
0,77th Street,2010,15.829451,34679.964508
1,77th Street,2011,12.154545,42531.909091
2,77th Street,2012,12.490909,42105.818182
3,77th Street,2013,13.745455,41582.090909
4,77th Street,2014,13.481818,40561.181818
...,...,...,...,...
247,Wilshire,2017,5.857143,74650.285714
248,Wilshire,2018,5.185714,81163.285714
249,Wilshire,2019,4.628571,84176.142857
250,Wilshire,2020,10.662999,77138.421389


In [11]:
crime['DATE_OCC'] = pd.to_datetime(crime['DATE_OCC'], format='%d/%m/%Y')
crime['Year'] = crime['DATE_OCC'].dt.year
crime = crime.astype({'Year':'int64'})
crime = crime.dropna()
crime['zipcode'] = crime.loc[:, ['zipcode']].applymap(np.int64)
crime.head()

,DR_NO,DATE_RPTD,DATE_OCC,TIME_OCC,AREA_NAME,CRM_1_SEVERITY,VICT_AGE,VICT_SEX,PREMIS_DESC,WEAPON_DESC,STATUS_DESC,LOCATION,LAT,LON,VICTIM_RACE,SEVERITY,crime_type,Year,zipcode
0,1307355,2010-02-20,2010-02-20,13:50:00,Newton,2,48,M,SINGLE FAMILY DWELLING,UNKNOWN WEAPON/OTHER WEAPON,Adult Arrest,300 E GAGE AV,33.9825,-118.2695,Hispanic/Latin/Mexican,2,Vandalism,2010,90003
1,11401303,2010-09-13,2010-09-12,00:45:00,Pacific,3,0,M,STREET,UNKNOWN WEAPON/OTHER WEAPON,Invest Cont,SEPULVEDA BL,33.9599,-118.3962,White,3,Arson/Embezzlement,2010,90045
2,70309629,2010-08-09,2010-08-09,15:15:00,Newton,1,0,M,ALLEY,UNKNOWN WEAPON/OTHER WEAPON,Invest Cont,1300 E 21ST ST,34.0224,-118.2524,Hispanic/Latin/Mexican,1,Trespass/Sex_offense,2010,90011
3,90631215,2010-01-05,2010-01-05,01:50:00,Hollywood,2,47,F,STREET,HAND GUN,Invest Cont,CAHUENGA BL,34.1016,-118.3295,White,3,Vandalism,2010,90028
4,100100501,2010-01-03,2010-01-02,21:00:00,Central,9,47,F,ALLEY,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",Invest Cont,8TH ST,34.0387,-118.2488,Hispanic/Latin/Mexican,9,Manslaughter/Homicide,2010,90014


In [12]:
# get average number of crimes per month for each area/year combination, 2021 only has 5 months
crimes = crime.groupby(['AREA_NAME','Year'])[['DR_NO']].count().reset_index()
crimes['Crimes_per_month'] = crimes.apply(lambda row: row['DR_NO']/12 if row['Year']<2021 else row['DR_NO']/5, axis=1)
crimes.drop(columns=['DR_NO'], inplace = True)
crimes

,AREA_NAME,Year,Crimes_per_month
0,77th Street,2010,1203.333333
1,77th Street,2011,1187.333333
2,77th Street,2012,1186.916667
3,77th Street,2013,1145.083333
4,77th Street,2014,1170.833333
...,...,...,...
247,Wilshire,2017,822.333333
248,Wilshire,2018,894.500000
249,Wilshire,2019,866.833333
250,Wilshire,2020,758.833333


In [13]:
# combine into one dataframe
regression = pop.merge(income, on = ['AREA_NAME','Year'])
regression = regression.merge(crimes, on = ['AREA_NAME','Year'])
regression

,AREA_NAME,Year,Population,Unemployment_rate,Median_household_income,Crimes_per_month
0,77th Street,2010,620172.0,15.829451,34679.964508,1203.333333
1,77th Street,2011,532941.0,12.154545,42531.909091,1187.333333
2,77th Street,2012,532445.0,12.490909,42105.818182,1186.916667
3,77th Street,2013,536127.0,13.745455,41582.090909,1145.083333
4,77th Street,2014,539384.0,13.481818,40561.181818,1170.833333
...,...,...,...,...,...,...
247,Wilshire,2017,192886.0,5.857143,74650.285714,822.333333
248,Wilshire,2018,190217.0,5.185714,81163.285714,894.500000
249,Wilshire,2019,186420.0,4.628571,84176.142857,866.833333
250,Wilshire,2020,218293.0,10.662999,77138.421389,758.833333


In [14]:
regression.groupby('Year')[['Population']].sum()

,Population
Year,
2010,9818599.0
2011,8468667.0
2012,8512029.0
2013,8564162.0
2014,8637017.0
2015,8693668.0
2016,8716469.0
2017,8762364.0
2018,8756467.0


In [15]:
regression.to_csv('regression_new.csv', index=False)